# SQRA with Lennar Jones Clusters - Convergence analysis

## The Setup
We start with a (1) simulation of a trajectory of the Lennard Jones cluster dynamics for 3 Particles in 2-Space.
The resulting points will be used for a subsampling procedure to obtain the cells for the SQRA.
There are two options:
- The Picking algorith: \
	Pick iteratively the point farthest from all previously picked points. For the adjacency matrix required for the SQRA we use a heuristic to match an expected average number of neighbours by a distance threshold. The Volumes and Areas of the cells are assumed to be uniform.

- Sparse Boxes: \
	We cover the trajectory with a grid of regular boxes. In this regime the adjacency is clear and volumes and areas are constant.
	For the estimation of the (assumedly constant) potential in each box we use the minimum of the correspoding samples.

### Postprocessing of the SQRA:
In order to obtain a 'nice' generator matrix we prune states with outbound rates above a certain thresholds and afterwards (unconnected) states without incoming rates.

## Committor computation
Based on the relative angles of the 3 particles we classify the states into the states: right oriented, left oriented, unoriented.
We then compute the committor function between the left and right orientations.

## Convergence analysis
In order to analyse the convergence of the SQRA we compute the committors for different levels of the sparse box discretization.
We then compute the distances of the committors (of different resolutions) with the finest committor.

For the distance we use the MSE on the support of the finest discretization.

In [4]:
using Revise
using Sqra
using Plots
using Arpack
using LinearAlgebra, Random
plotly(fmt=:png)

Plots.PlotlyBackend()

# Simulation

We start by simulation a long trajectory to explore the state space.
We use these samples as a basis for either subsampling with the picking algorithm (`:voronoi`)
or for selecting the sparse boxes for a given discretization level (`ncells` in each direction)

In [33]:
using Profile
Profile.init()
Profile.init(10_000_000, 0.1)

In [34]:
#Random.seed!(0)
@profile sim = Sqra.run(Sqra.Simulation(nsteps=2_000_000, sigma=.5, maxdelta=0.01, seed=rand(UInt)))

Euler Maruyama simulation100%|██████████████████████████| Time: 0:00:46
┌ Info: saved new entry
└ @ Sqra /home/htc/bzfsikor/code/Sqra.jl/src/permadict.jl:27


Sqra.Simulation
  x0: Array{Float64}((6,)) [0.19920158482463968, 0.13789462153196408, -0.1709575705426315, 0.0784533378749835, 0.06778720715969005, -0.2112155752270007]
  epsilon: Int64 1
  r0: Float64 0.3333333333333333
  harm: Int64 1
  sigma: Float64 0.5
  dt: Float64 0.001
  nsteps: Int64 2000000
  maxdelta: Float64 0.01
  seed: UInt64 0x1c9f879269bde138
  x: Array{Float64}((6, 2000001)) [0.19920158482463968 0.21829736041164463 … -0.15702675522255677 -0.15498278790861944; 0.13789462153196408 0.11936988751377531 … -0.17577924583960525 -0.1981373259931556; … ; 0.06778720715969005 0.06196063372663429 … -0.23601623890872017 -0.23651442806580264; -0.2112155752270007 -0.2236839921108189 … -0.5436346948732401 -0.5492257411685667]
  u: Array{Float64}((2000001,)) [-2.855863452445878, -2.6897250754717694, -2.566919907033898, -2.63193392676249, -2.6275558675518287, -2.574937520175813, -2.240676948524751, -2.29264110352562, -2.512976412464236, -2.675950653103219  …  -2.279607262266034, -2.3136

In [41]:
Profile.print(maxdepth=15, mincount=40)

Overhead ╎ [+additional indent] Count File:Line; Function
   ╎509  @Base/task.jl:411; (::IJulia.var"#15#18")()
   ╎ 509  ...lia/src/eventloop.jl:8; eventloop(socket::ZMQ.Socket)
   ╎  509  @Base/essentials.jl:706; invokelatest
   ╎   509  @Base/essentials.jl:708; #invokelatest#2
   ╎    509  .../execute_request.jl:67; execute_request(socket::ZMQ.So...
   ╎     509  ...SoftGlobalScope.jl:65; softscope_include_string(m::M...
   ╎    ╎ 509  @Base/loading.jl:1094; include_string(mapexpr::typ...
   ╎    ╎  509  @Base/boot.jl:360; eval
   ╎    ╎   509  ...ze/src/Memoize.jl:61; run(params::Sqra.Simulation)
   ╎    ╎    508  .../src/permadict.jl:25; get!(f::Sqra.var"#13#15"{Sq...
   ╎    ╎     508  ...e/src/Memoize.jl:62; #13
   ╎    ╎    ╎ 461  .../lennardjones.jl:95; var"##run_unmemoized"(par...
   ╎    ╎    ╎  461  ...ulermaruyama.jl:29; (::Sqra.var"#eulermaruyam...
   ╎    ╎    ╎   56   ...ulermaruyama.jl:41; eulermaruyama(x0::Vector...
   ╎    ╎    ╎    56   ...src/gradient.jl:35; gradien

In [ ]:
r = Sqra.discretize(Sqra.SpBoxDiscretisation(ncells=15, prune=Inf), sim)

In [ ]:
plot(sum(r.Q .> 0, dims=1)|>vec|>sort, title="number of neighbours")

In [ ]:
plot(-diag(r.Q), yaxis=:log, title="distribution of outbound rates")

In [ ]:
plot(r.u)

In [ ]:
plot((sort(-diag(r.Q))), yaxis=:log)

In [ ]:
println("size of system")
length(r.Q.nzval), size(r.Q)

In [ ]:
@time c=Sqra.committor(r, Sqra.gmres, precondition=false, maxiter=10000)



# Analysis of committor solver convergence

We compare different solvers for the linear system, as well as the solutions of a more pruned system.
Pruning is mainly there to enable solution of the linear committor system.
On the other hand iterative solvers might maybe help to compute ill conditioned systems?
After all the large outbound rates should not play a role in the computation.
We might as well try preconditioning with the diagonal rates!

In [ ]:
c = @time Sqra.committor(r, maxiter=10000)

In [ ]:
@show beta = Sqra.sigma_to_beta(sim.sigma)

stat = stat / sum(stat)
plot(stat)

In [ ]:
A, b = Sqra.committor_system(r.Q, Sqra.classify(r.picks))

sqrt(sum(abs2, (A*c - b)[abs.(diag(r.Q))  .< 1e5]))



# Taking a look at committor solution

In [ ]:
# plot states with a committor value in the defined range
transind = rand(findall((0.4 .<c.<0.6)), 100)
plot()
Sqra.plot_triangles!(Sqra.normalform(r.picks[:, transind]))

In [ ]:
# plot the rotated and translated normal form of the LJ clusters and color by committor

plot()
plotint = 1:30:size(r.picks,2)
color = c
let points = r.picks[:, plotint], col = c[plotint]
#let points = r.picks[:, pinds], col = x
	@time Sqra.plot_trajectories(Sqra.normalform(points), alpha=0.3, marker_z=col) |> display;
end 

In [ ]:
# plot the JL cluster states in original coordinates
plot();
@time Sqra.plot_trajectories(r.picks[:,plotint], alpha=0.1) |> display;
#plot_trajectories(r.x[:,1:end], alpha=0.01, markersize=0.5)

In [ ]:
plot(c|>sort, title="distribution of committor values")

# Convergence of committors


In [ ]:
runs = []

levels=3:14

for ncells in levels
	r = Sqra.discretize(Sqra.SpBoxDiscretisation(ncells=ncells), sim)
	c = Sqra.committor(r)
	push!(runs, (c=c, r=r))
end

In [ ]:
@timed arst=1

In [ ]:
_c = runs[end].c
_carts = runs[end].r.cartesians
_ncells = runs[end].r.ncells

conv = map(runs) do run
	c = run.c
	carts = run.r.cartesians
	ncells = run.r.ncells

	@time Sqra.sp_mse(c, _c, carts, _carts, ncells,  _ncells)
end

In [ ]:
plot(levels[1:end-1],conv[1:end-1], yaxis=:log, xaxis=:log)

In [ ]:
string(hash(_c))

In [ ]:
]resolve

In [ ]:
@time Sqra.batch(levels=3:5);

In [ ]:
Sqra.jldopen("3.jld2") |>close

# Convergence of picking, Part 1
We analyse how the number of boxes increases with growing sample size and resolution

In [ ]:
]add Parameters

In [ ]:
plot()
for n in [3,4,5,6,8,10]
    _, _, order = Sqra.sparseboxpick(r.x, n, ones(size(r.x,2)), r.boundary)
    plot!(order, 1:length(order), )
    accel =  (order[end]-order[end-10]) / 10
    #println(" $(length(order)/size(r.x, 2)  / accel * 100) % verbesserung pro prozent ")
end
plot!(legend=false, xlabel="# samples", ylabel="# boxes", yaxis=:log, xaxis=:log)

# --- OLD SNIPPETS --- #

# Generator preprocessing

In [ ]:
cutoff = 3
prune = Inf

Q, pinds = Sqra.prune_Q(r.Q,Inf)

Q.nzval[Q.nzval.>cutoff] .= cutoff
Q = Sqra.fixdiagonal(Q)

In [ ]:
plot((Q.nzval|>sort))

# Committor convergence

In [ ]:
Sqra.solve_committor(Q, r.classes[pinds])

# Spectrum

In [ ]:
@time evals, evecs = eigs(Q, which=:LR, maxiter=10000, tol=10)

In [ ]:
evals

In [ ]:
plot(evecs[:,:].|>real)
#plot!(r.classes, alpha=0.5)

# Cell analysis

In [ ]:
prob = argmin(diag(r.Q))

In [ ]:
@show r.Q[prob,:]
neighs = r.Q[prob,:].nzind;

In [ ]:
@show r.us[prob]
r.us[neighs]

In [ ]:
plot(); Sqra.plot_trajectories(r.picks[:,prob])
Sqra.plot_triangles!(r.picks[:, neighs], color=:black, alpha=0.8, legend=false)

# Spectrum again

In [ ]:
evals, evecs = eigs(r.Q, which=:SM, nev=6)#, check=0, maxiter=1000, tol=0)
evecs = real.(evecs)
evals

In [ ]:
evecs

In [ ]:
plot(real.(evecs))

In [ ]:
step=100
for evec in 1:length(evals)

    col = real.(evecs[:,evec])[1:step:end]
    data = center[:,1:step:end]
    plot();
    plot_trajectories(normalform(data), alpha=0.3, marker_z=col, clims=(-1,1).*std(col), seriescolor=:bwr) |> display

    scatter(data[1,:], data[2,:], alpha=0.3, marker_z=col, clims=(-1,1).*std(col), seriescolor=:bwr) |> display
end